In [21]:
from dataclasses import dataclass
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import pprint


@dataclass
class Option:
    id: str
    strike: float
    expiry_date: datetime
    entry_date: datetime | None = None
    exit_date: datetime | None = None


def generate_options(
    strikes: np.ndarray,
    expiry_dates: list[datetime],
    underlying_stock_ticker: str = "XYZ",
) -> list[Option]:
    """
    Option id follows this: https://polygon.io/blog/how-to-read-a-stock-options-ticker
    """
    options = []
    for strike in strikes:
        for expiry_date in expiry_dates:
            expiry_date_str = expiry_date.strftime("%y%m%d")
            option_type = "C" if strike >= 0 else "P"
            strike_price_str = f"{int(abs(strike) * 1000):08d}"
            option_id = f"{underlying_stock_ticker}{expiry_date_str}{option_type}{strike_price_str}"
            option = Option(id=option_id, strike=strike, expiry_date=expiry_date)
            options.append(option)
    return options


def should_rollover(option: Option, current_date: datetime) -> bool:
    """Determine if the held option should be rolled over based on the current date."""
    return (option.expiry_date - current_date).days <= 10


def find_next_option(
    available_options: list[Option], current_date: datetime, current_market_price: float
) -> Option:
    """Find the next option to rollover to based on the criteria:
    1. Has at least 30 days before expiry.
    2. Is nearest to the current day.
    3. Is ATM (At-The-Money).
    """
    # Filter options that have at least 30 days before expiry
    valid_options = [
        option
        for option in available_options
        if (option.expiry_date - current_date).days >= 30
    ]
    if not valid_options:
        return None

    # Finding the option that is nearest to being At-The-Money (ATM)
    atm_option = min(
        valid_options,
        key=lambda x: (
            abs(x.strike - current_market_price),
            (x.expiry_date - current_date).days,
        ),
    )
    return atm_option


def rollover_options(
    available_options: list[Option], prices_df: pd.DataFrame
) -> list[Option]:
    prices_df["date"] = pd.to_datetime(prices_df["date"])

    rolled_over_options = []
    current_option = None

    for i, row in prices_df.iterrows():
        # current_date = datetime.strptime(row["date"], "%Y-%m-%d")
        current_date = row["date"]
        current_market_price = row["price"]

        if current_option is None or should_rollover(current_option, current_date):
            next_option = find_next_option(
                available_options, current_date, current_market_price
            )
            if next_option and (
                current_option is None or next_option.id != current_option.id
            ):
                if current_option:
                    current_option.exit_date = current_date
                next_option.entry_date = current_date

                rolled_over_options.append(next_option)
                print(
                    f"{current_date.strftime('%y-%m-%d')} {current_market_price} => Rolled over to option {next_option.strike}@{next_option.expiry_date.strftime('%y%m%d')}"
                )
                current_option = next_option  # Update the held option
            elif not next_option:
                print("No suitable option found to rollover to.")
                break  # Exit the loop if no suitable next option is found

    return rolled_over_options


if __name__ == "__main__":
    # Example usage
    generate_new_prices = False
    output_path = "generated/XYZ"

    if generate_new_prices:
        prices = np.array([100, 105, 110, 115, 120])
        start_date = datetime(2023, 1, 1)
        end_date = datetime(2023, 12, 31)
        strikes = np.arange(
            (min(prices) // 5 - 2) * 5, (max(prices) // 5 + 3) * 5 + 1, 5
        )
        expiry_dates = [
            start_date + timedelta(days=days) for days in range(30, 361, 30)
        ]
    else:
        stock_prices_df = pd.read_csv(f"{output_path}/XYZ_stock.csv")

    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)

    prices = stock_prices_df["price"]
    strikes = np.arange((min(prices) // 5 - 2) * 5, (max(prices) // 5 + 3) * 5 + 1, 5)
    expiry_dates = [
        datetime(2023, 1, 1) + timedelta(days=days) for days in range(30, 361, 30)
    ]
    available_options = generate_options(strikes, expiry_dates, "XYZ")
    rolled_over_options = rollover_options(available_options, stock_prices_df)

    print("-" * 50)
    # pprint.pp(available_options)
    pprint.pp(rolled_over_options)

23-01-01 100.8949518369175 => Rolled over to option 100.0@230131
23-01-21 106.74590553727678 => Rolled over to option 105.0@230302
23-02-20 107.06462789942132 => Rolled over to option 105.0@230401
23-03-22 116.4191754741632 => Rolled over to option 115.0@230501
23-04-21 115.9608959798998 => Rolled over to option 115.0@230531
23-05-21 110.0537346410046 => Rolled over to option 110.0@230630
23-06-20 100.75549054372834 => Rolled over to option 100.0@230730
23-07-20 96.64018795355034 => Rolled over to option 95.0@230829
23-08-19 100.20525159434672 => Rolled over to option 100.0@230928
23-09-18 103.61027706022878 => Rolled over to option 105.0@231028
23-10-18 111.00237621297448 => Rolled over to option 110.0@231127
23-11-17 112.15358977323704 => Rolled over to option 110.0@231227
No suitable option found to rollover to.
--------------------------------------------------
[Option(id='XYZ230131C00100000',
        strike=100.0,
        expiry_date=datetime.datetime(2023, 1, 31, 0, 0),
        e